In [61]:
%load_ext autoreload
%autoreload 2
import tensorflow_quantum as tfq
from utilities.variational import VQE
from utilities.qmodels import *
from utilities.evaluator import Evaluator
from utilities.idinserter import IdInserter
from utilities.unitary_killer import UnitaryMurder
from utilities.simplifier import Simplifier
import cirq
import numpy as np

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
#indexed_circuit = [5,4,2,4,3,5,3,0,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
indexed_circuit =[5, 3, 5, 3, 4, 2, 4, 1, 3, 1, 3, 5, 0, 2, 4]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)), energy)


tf.Tensor([-7.9999995], shape=(1,), dtype=float32) tf.Tensor(-7.999999, shape=(), dtype=float32)


In [63]:
cirq.resolve_parameters(circuit,resolver)

(0, 0): ───Rx(0.725π)───Rz(1.54π)─────Rx(-1.45π)────────────────X───────────────X─────────────────────────────@───Rz(0.228π)───Rx(0.671π)───
                                                                │               │                             │
(0, 1): ───Rx(-1.91π)───Rz(-0.488π)───Rx(1.54π)────Rz(-0.27π)───@───Rz(1.54π)───@───Rz(-0.366π)───Rx(1.22π)───X─────────────────────────────

In [64]:
i,r,po=simplifier.reduce_circuit(indexed_circuit,resolver,index_to_symbols)

5
cnt_rule 5
6
cnt_rule 6
3
cnt_rule 3
6
cnt_rule 6
2
cnt_rule 2


In [65]:
nu=vqe_handler.give_unitary(i,r)
nu

(0, 0): ───Rx(0.725π)───Rz(1.54π)───Rx(-1.45π)───@───Rz(0.228π)───Rx(0.671π)───
                                                 │
(0, 1): ───Rx(1.55π)────Rz(-π)──────Rx(1.22π)────X─────────────────────────────

In [66]:
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-8.], dtype=float32)>

In [182]:

vqe_handler = VQE(n_qubits=2, problem_config={"problem":"XXZ","g":1,"J":2})
iid_inserter = IdInserter(n_qubits = vqe_handler.n_qubits, selector_temperature=40, epsilon=0.1)
simplifier = Simplifier(n_qubits=vqe_handler.n_qubits,testing=True)

#indexed_circuit = [vqe_handler.number_of_cnots + k for k in range(vqe_handler.n_qubits, 2*vqe_handler.n_qubits )]
indexed_circuit = [5,5,5,1,5,5,5]
#indexed_circuit = [5,3,3,5,4,0,5,5,3,3,5,3,5]
circuit, symbols, index_to_symbols = vqe_handler.give_circuit(indexed_circuit)
resolver = {s:r for s,r in zip(symbols, np.random.randn(len(symbols)))}
#energy, resolver, history = vqe_handler.vqe(indexed_circuit)

model = QNN(symbols=list(resolver.keys()), observable=vqe_handler.observable)
print(model(cirq.resolve_parameters(circuit,resolver)))


tf.Tensor([-1.035732], shape=(1,), dtype=float32)


In [183]:
i,r,po=simplifier.simplify_step(indexed_circuit,resolver,index_to_symbols)

4
here:  ['th_1', 'th_2']
0.8779985591597353 th_0
4
here:  ['th_4', 'th_5']
1.9401020044321542 th_1


In [184]:
nu=vqe_handler.give_unitary(i,r)
nu
model(nu)

<tf.Tensor: shape=(1,), dtype=float32, numpy=array([-1.0357327], dtype=float32)>